In [1]:
import scanpy as sc

In [2]:
import pandas as pd

In [ ]:
#去除其他染色体上的基因和一些样本以及细胞类型
adata = sc.read_h5ad('/CIMA/scRNA_Data/NatualCohort_All_Annotation_Final_reUMAP.h5ad')

In [4]:
adata.X = adata.layers['counts']

In [18]:
adata_B_use_for_dynamic = adata[adata.obs["celltype_l1"] == "B"]
adata_B_use_for_dynamic = adata_B_use_for_dynamic[adata_B_use_for_dynamic.obs["celltype_l2"] != 'Total_Plasma']

In [19]:
adata_B_use_for_dynamic.write_h5ad('/CIMA/Data/dynamic/single_cell/B_counts.h5ad')

In [20]:
adata_Mono_use_for_dynamic = adata[adata.obs["celltype_l2"] == "Monocyte"]

In [21]:
adata_Mono_use_for_dynamic.write_h5ad('/CIMA/Data/dynamic/single_cell/Monocyte_counts.h5ad')

In [22]:
#获取mRNA_PC
del adata_B_use_for_dynamic.uns
del adata_B_use_for_dynamic.obsp
del adata_B_use_for_dynamic.obsm

sample_with_genomics = pd.read_table('../Data/413sample.txt')
#去除没有wgs信息的样本
adata_B_use_for_dynamic = adata_B_use_for_dynamic[adata_B_use_for_dynamic.obs['sample'].isin(sample_with_genomics['FID']),:]

sc.pp.normalize_total(adata_B_use_for_dynamic, target_sum=1e4)

sc.pp.log1p(adata_B_use_for_dynamic)

sc.pp.highly_variable_genes(adata_B_use_for_dynamic)

adata_B_use_for_dynamic = adata_B_use_for_dynamic[:, adata_B_use_for_dynamic.var.highly_variable]

sc.pp.regress_out(adata_B_use_for_dynamic, ["total_counts", "pct_counts_mt"])

sc.pp.scale(adata_B_use_for_dynamic, max_value=10)

sc.tl.pca(adata_B_use_for_dynamic, svd_solver="arpack")

mRNA_pca_matrix_B = pd.DataFrame(adata_B_use_for_dynamic.obsm['X_pca'])

mRNA_pca_matrix_B.index = adata_B_use_for_dynamic.obs.index

mRNA_pca_matrix_B.columns =  [f"mRNA_PC{i+1}" for i in range(50)]

mRNA_pca_matrix_B.to_csv('/CIMA/Data/dynamic/single_cell/mRNA_pc_for_B.csv')

/home/huangzhuoli/mambaforge/envs/scPAFA_env/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/huangzhuoli/mambaforge/envs/scPAFA_env/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:641: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [23]:
#获取mRNA_PC
del adata_Mono_use_for_dynamic.uns
del adata_Mono_use_for_dynamic.obsp
del adata_Mono_use_for_dynamic.obsm

sample_with_genomics = pd.read_table('../Data/413sample.txt')
#去除没有wgs信息的样本
adata_Mono_use_for_dynamic = adata_Mono_use_for_dynamic[adata_Mono_use_for_dynamic.obs['sample'].isin(sample_with_genomics['FID']),:]

sc.pp.normalize_total(adata_Mono_use_for_dynamic, target_sum=1e4)

sc.pp.log1p(adata_Mono_use_for_dynamic)

sc.pp.highly_variable_genes(adata_Mono_use_for_dynamic)

adata_Mono_use_for_dynamic = adata_Mono_use_for_dynamic[:, adata_Mono_use_for_dynamic.var.highly_variable]

sc.pp.regress_out(adata_Mono_use_for_dynamic, ["total_counts", "pct_counts_mt"])

sc.pp.scale(adata_Mono_use_for_dynamic, max_value=10)

sc.tl.pca(adata_Mono_use_for_dynamic, svd_solver="arpack")

mRNA_pca_matrix_Mono = pd.DataFrame(adata_Mono_use_for_dynamic.obsm['X_pca'])

mRNA_pca_matrix_Mono.index = adata_Mono_use_for_dynamic.obs.index

mRNA_pca_matrix_Mono.columns =  [f"mRNA_PC{i+1}" for i in range(50)]

mRNA_pca_matrix_Mono.to_csv('/CIMA/Data/dynamic/single_cell/mRNA_pc_for_Mono.csv')

/home/huangzhuoli/mambaforge/envs/scPAFA_env/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/huangzhuoli/mambaforge/envs/scPAFA_env/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:641: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [33]:
import numpy as np

In [39]:
sig_eGene_B = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/B/20250122_eGene_use_for_down_stream.csv',index_col=0)

sig_eGene_B = sig_eGene_B['phenotype_id'].unique()

sig_eGene_B = np.array([f"B_xxx_{x}" for x in sig_eGene_B])

In [41]:
pd.Series(sig_eGene_B).to_csv('/CIMA/Script/B_cell_dynamic_eGene.txt',sep='\t',index=False, header=False)

In [42]:
sig_eGene_Mono = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/Monocyte/20250122_eGene_use_for_down_stream.csv',index_col=0)

sig_eGene_Mono = sig_eGene_Mono['phenotype_id'].unique()

sig_eGene_Mono = np.array([f"Monocyte_xxx_{x}" for x in sig_eGene_Mono])

In [44]:
len(sig_eGene_Mono)

3931

In [45]:
pd.Series(sig_eGene_Mono).to_csv('/CIMA/Script/Mono_cell_dynamic_eGene.txt',sep='\t',index=False, header=False)

In [ ]:
B_mRNA_pc = pd.read_csv('/CIMA/Data/dynamic/single_cell/mRNA_pc_for_B.csv',index_col=0)
B_mRNA_pc = B_mRNA_pc.iloc[:,0:5]

B_pseudotime = pd.read_csv('/CIMA/Science返修/scTour/B_ptime.csv',index_col = 0)
B_pseudotime = B_pseudotime.loc[B_mRNA_pc.index,:]

#取反
B_pseudotime['ptime'] = 1- B_pseudotime['ptime'] 

adata_B_use_for_dynamic = sc.read_h5ad('/CIMA/Data/dynamic/single_cell/B_counts.h5ad')
adata_B_use_for_dynamic = adata_B_use_for_dynamic[B_mRNA_pc.index]

useful_obs = adata_B_use_for_dynamic.obs.loc[:,['sample','total_counts','pct_counts_mt']]
sum(adata_B_use_for_dynamic.obs.index == useful_obs.index)

merge_df = pd.concat([B_mRNA_pc,B_pseudotime,useful_obs],axis=1)
keep_index = merge_df.index
sample_info_new = pd.read_csv('/CIMA/Data/sample_info_new.csv')
sum(merge_df['sample'].isin(sample_info_new['BGE_name']))

merge_df = pd.merge(
    merge_df,
    sample_info_new,
    left_on='sample',
    right_on='BGE_name',
    how='left'
)
merge_df.index = keep_index

merge_df.to_csv('/CIMA/Data/dynamic/single_cell/B_cov_df.csv')

sig_eGene = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/B/20250122_eGene_use_for_down_stream.csv',index_col=0)
adata_B_use_for_dynamic = adata_B_use_for_dynamic[:,sig_eGene['phenotype_id'].unique()]
dense_data = adata_B_use_for_dynamic.to_df()
dense_data['sample'] = adata_B_use_for_dynamic.obs['sample']
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')
genotype_df = genotype_df.T
genotype_df['sample'] = genotype_df.index

for index, row in sig_eGene.iterrows():
    phenotype = row['phenotype_id']
    variant = row['variant_id']

    df1 = dense_data[[phenotype,'sample']]
    df2 = genotype_df[[variant,'sample']]

    df3 = pd.merge(
        df1,
        df2,
        on='sample',
        how='left'
    )

    df3.index = df1.index
    df3.columns = ['E','sample','G']
    df3.to_csv(f'/CIMA/Data/dynamic/single_cell/B/{phenotype}.csv')

In [ ]:
Mono_mRNA_pc = pd.read_csv('/CIMA/Data/dynamic/single_cell/mRNA_pc_for_Mono.csv',index_col=0)
Mono_mRNA_pc = Mono_mRNA_pc.iloc[:,0:5]

Mono_pseudotime = pd.read_csv('/CIMA/Science返修/scTour/Mono_ptime.csv',index_col = 0)
Mono_pseudotime = Mono_pseudotime.loc[Mono_mRNA_pc.index,:]
#取反
Mono_pseudotime['ptime'] = 1-Mono_pseudotime['ptime'] 

adata_Mono_use_for_dynamic = sc.read_h5ad('/CIMA/Data/dynamic/single_cell/Monocyte_counts.h5ad')
adata_Mono_use_for_dynamic = adata_Mono_use_for_dynamic[Mono_mRNA_pc.index]

useful_obs = adata_Mono_use_for_dynamic.obs.loc[:,['sample','total_counts','pct_counts_mt']]
sum(adata_Mono_use_for_dynamic.obs.index == useful_obs.index)

merge_df = pd.concat([Mono_mRNA_pc,Mono_pseudotime,useful_obs],axis=1)
keep_index = merge_df.index
sample_info_new = pd.read_csv('/CIMA/Data/sample_info_new.csv')
sum(merge_df['sample'].isin(sample_info_new['BGE_name']))

merge_df = pd.merge(
    merge_df,
    sample_info_new,
    left_on='sample',
    right_on='BGE_name',
    how='left'
)
merge_df.index = keep_index

merge_df.to_csv('/CIMA/Data/dynamic/single_cell/Monocyte_cov_df.csv')

sig_eGene = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/Monocyte/20250122_eGene_use_for_down_stream.csv',index_col=0)
adata_Mono_use_for_dynamic = adata_Mono_use_for_dynamic[:,sig_eGene['phenotype_id'].unique()]
dense_data = adata_Mono_use_for_dynamic.to_df()
dense_data['sample'] = adata_Mono_use_for_dynamic.obs['sample']
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')
genotype_df = genotype_df.T
genotype_df['sample'] = genotype_df.index

for index, row in sig_eGene.iterrows():
    phenotype = row['phenotype_id']
    variant = row['variant_id']

    df1 = dense_data[[phenotype,'sample']]
    df2 = genotype_df[[variant,'sample']]

    df3 = pd.merge(
        df1,
        df2,
        on='sample',
        how='left'
    )

    df3.index = df1.index
    df3.columns = ['E','sample','G']
    df3.to_csv(f'/CIMA/Data/dynamic/single_cell/Monocyte/{phenotype}.csv')